In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1Ai6MxOfcX1s3XFnM-rGmw-MmfctdBvBe", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))

In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

# Patches & Embeddings: Teaching a Transformer to See

*Part 1 of the Vizuara series on Vision Transformers from Scratch*
*Estimated time: 45 minutes*

In this notebook, we will build the **input pipeline** for a Vision Transformer (ViT) completely from scratch. By the end, you will understand how an image gets converted into a sequence of tokens — the same kind of sequence a language model would process.

**What we will build today:**
- Patch extraction from images
- Flattening and linear projection (patch embeddings)
- The [CLS] classification token
- Learnable position embeddings

**Series roadmap:**
- **Notebook 1 (this one):** Patches & Embeddings
- **Notebook 2:** Multi-Head Self-Attention for Images
- **Notebook 3:** The Full ViT — Assembly, Training, and Evaluation

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://pods.vizuara.ai/courses/vision-transformers-from-scratch/practice/1/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


## Setup

In [ ]:
# Install any missing dependencies (Colab-friendly)
!pip install -q torch torchvision matplotlib numpy scikit-learn

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
#@title 🎧 Listen: Why Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_why_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 1: Why Does This Matter?

The **Vision Transformer (ViT)** rests on a beautifully simple idea: **an image is just a sentence made of patches**.

Natural Language Processing had its revolution when Transformers replaced recurrent networks. Words get turned into tokens, tokens get embedded into vectors, and attention does the rest. ViT asks: *what if we did exactly the same thing with images?*

Instead of word tokens, we have **patch tokens**. Instead of a vocabulary embedding table, we have a **linear projection layer**. Instead of positional encoding over a 1D sequence, we have **position embeddings that must somehow capture 2D spatial structure**.

This notebook builds that entire input pipeline. By the end, you will see something remarkable: even with completely random, untrained position embeddings, the cosine similarity matrix already hints at structure. After training, these embeddings **learn 2D spatial relationships on their own** — without anyone telling the model that images are 2D.

This is the foundation. Everything else in the Vision Transformer — attention, classification, training — builds directly on what we construct today.

Let us get a quick sense of the numbers before diving in.

In [ ]:
#@title 🎧 Listen: Key Numbers
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_key_numbers.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# Quick preview: the key numbers for ViT-Base
print("Vision Transformer Input Pipeline — Key Numbers")
print("=" * 50)
print(f"Image size:        224 x 224 pixels")
print(f"Patch size:        16 x 16 pixels")
print(f"Number of patches: (224/16)^2 = {(224//16)**2}")
print(f"Patch dimension:   16 x 16 x 3 = {16*16*3}")
print(f"Sequence length:   {(224//16)**2} patches + 1 [CLS] = {(224//16)**2 + 1}")
print(f"\nFor CIFAR-10 (our implementation):")
print(f"Image size:        32 x 32 pixels")
print(f"Patch size:        4 x 4 pixels")
print(f"Number of patches: (32/4)^2 = {(32//4)**2}")
print(f"Patch dimension:   4 x 4 x 3 = {4*4*3}")
print(f"Sequence length:   {(32//4)**2} patches + 1 [CLS] = {(32//4)**2 + 1}")

In [ ]:
#@title 🎧 Listen: Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 2: Building Intuition

### The Reading Analogy

Consider three ways to process information:

**Reading a book:** You consume it **word by word**, left to right, line by line. Each word is a discrete token. This is how Transformers process language.

**Looking at a painting with a magnifying glass:** You slide a small window across the canvas, examining local neighborhoods of pixels. The window shares its parameters everywhere it goes. This is how **Convolutional Neural Networks** process images — local receptive fields, shared weights, sliding across the image.

**Looking at a painting through a grid:** You divide the canvas into a grid of squares, examine each square as a complete unit, and then reason about how they relate to each other. This is how a **Vision Transformer** processes images.

The ViT approach is radical in its simplicity. No convolutions. No pooling. No inductive bias about local neighborhoods. Just: chop the image into patches, embed them, and let attention figure out the rest.

### Thought Experiment: The Shuffled Photo

Imagine you take a photograph and cut it into a 14x14 grid of squares. Now shuffle all 196 squares randomly. Could you still recognize what is in the photo?

Surprisingly, **mostly yes**. Each patch carries enough local information — a piece of fur, part of an eye, a section of sky — that you could probably identify the subject. Patches are information-rich units.

But could you **reconstruct** the original scene? Absolutely not. You would need to know **where each patch belongs**. This is exactly why we need **position embeddings** — they tell the Transformer where each patch came from in the original image.

> **Think About This:** If we treat each patch as a "word," what is the "vocabulary size" of our visual language? In English, the vocabulary is finite — maybe 50,000 words. But for image patches, every possible 16x16 arrangement of RGB pixels is a valid "word." The vocabulary is effectively **infinite**. This is why we cannot use a lookup table (like word embeddings) and must instead use a **learned linear projection** to map raw pixels into embedding space.

Let us quantify this. How large is the "vocabulary" of image patches compared to natural language?

In [ ]:
#@title 🎧 Listen: Visual Vocab
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_visual_vocab.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# How big is the visual "vocabulary"?
import math

# English vocabulary: ~50,000 common words
english_vocab = 50_000

# A 16x16 RGB patch: each pixel has 256 values per channel
# Total possible patches = 256^(16*16*3) — astronomically large
patch_pixels = 16 * 16 * 3  # 768 values
print(f"English vocabulary: ~{english_vocab:,} words")
print(f"Pixels per ViT patch: {patch_pixels}")
print(f"Possible patches: 256^{patch_pixels} ≈ 10^{patch_pixels * math.log10(256):.0f}")
print(f"\nThat is why we CANNOT use a lookup table!")
print(f"Instead, we learn a linear projection from pixel space to embedding space.")

In [ ]:
#@title 🎧 Listen: Math Patches
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_math_patches.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 3: The Mathematics

Let us walk through the math carefully. After every equation, we will translate it into concrete computation.

### 3.1 How Many Patches?

Given an image of height $H$ and width $W$, divided into non-overlapping patches of size $P \times P$:

$$N = \frac{H}{P} \times \frac{W}{P} = \left(\frac{H}{P}\right)^2 \quad \text{(for square images)}$$

**Computationally:** Divide the image height by the patch size and square it. For the standard ViT-Base configuration with 224x224 images and 16x16 patches:

$$N = \left(\frac{224}{16}\right)^2 = 14^2 = 196 \text{ patches}$$

For our CIFAR-10 implementation (32x32 images, 4x4 patches):

$$N = \left(\frac{32}{4}\right)^2 = 8^2 = 64 \text{ patches}$$

Each of these patches becomes one "token" in our sequence — analogous to one word in a sentence.

### 3.2 Flattening a Patch

Each patch is a small image of size $P \times P \times C$ (where $C$ is the number of channels — 3 for RGB). We flatten it into a single vector:

$$\mathbf{x}_p^i \in \mathbb{R}^{P^2 \cdot C}$$

**Computationally:** Take each 16x16x3 patch and unroll it into a single vector. The dimension is $16 \times 16 \times 3 = 768$. For our CIFAR-10 setup: $4 \times 4 \times 3 = 48$.

Think of it like this: you are reading all the pixel values of a patch from left to right, top to bottom, channel by channel, and writing them into one long list.

Let us verify these dimensions in code.

In [ ]:
# Verify patch dimensions for different configurations
configs = [
    ("ViT-Base (ImageNet)", 224, 16, 3),
    ("Our CIFAR-10 setup", 32, 4, 3),
    ("Toy example", 8, 4, 1),
]

print(f"{'Config':<25} {'Patches':>8} {'Patch Dim':>10} {'Seq Len':>8}")
print("-" * 55)
for name, img, p, c in configs:
    n = (img // p) ** 2
    dim = p * p * c
    print(f"{name:<25} {n:>8} {dim:>10} {n+1:>8}")

In [ ]:
#@title 🎧 Listen: Math Projection
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_math_projection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 3.3 Linear Projection (Patch Embedding)

The raw flattened patch vector has dimension $P^2 \cdot C$. We project it into the model's hidden dimension $D$ using a learnable weight matrix:

$$\mathbf{z}_0^i = \mathbf{x}_p^i \, \mathbf{E} + \mathbf{b}$$

where $\mathbf{E} \in \mathbb{R}^{(P^2 \cdot C) \times D}$ and $\mathbf{b} \in \mathbb{R}^D$.

**Computationally:** This is a single matrix multiplication — exactly `nn.Linear(patch_dim, embed_dim)`. It takes our 768-dimensional raw pixel vector and maps it into, say, a 512-dimensional embedding space. The weight matrix $\mathbf{E}$ is learned during training, so the model discovers which combinations of pixel values are important.

Does this remind you of something? In NLP, a word embedding layer maps a one-hot word vector into a dense embedding. Here, instead of one-hot vectors, we have raw pixel vectors — but the idea is identical.

### 3.4 The Full Input Sequence

The complete input to the Transformer is:

$$\mathbf{z}_0 = [\mathbf{x}_{\text{cls}};\; \mathbf{x}_p^1 \mathbf{E};\; \mathbf{x}_p^2 \mathbf{E};\; \ldots;\; \mathbf{x}_p^N \mathbf{E}] + \mathbf{e}_{\text{pos}}$$

where $\mathbf{x}_{\text{cls}} \in \mathbb{R}^D$ is the learnable [CLS] token and $\mathbf{e}_{\text{pos}} \in \mathbb{R}^{(N+1) \times D}$ are the position embeddings.

**Computationally:** Four steps:
1. Flatten each patch into a vector (reshape)
2. Project each vector through the linear layer (matrix multiply)
3. Prepend the [CLS] token to get $N+1$ tokens
4. Add position embeddings (element-wise addition)

The output shape is $(B, N+1, D)$ — a batch of sequences, each with $N+1$ tokens of dimension $D$.

In [ ]:
#@title 🎧 Listen: Toy Example
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_toy_example.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 3.5 Toy Example: Walking Through the Math

Let us trace through a tiny example to make this completely concrete.

**Setup:** 8x8 grayscale image, patch size 4.

In [ ]:
# Toy example: 8x8 grayscale image, patch size 4
H, W, C = 8, 8, 1
P = 4
D = 6  # tiny embedding dimension for illustration

# Number of patches
N = (H // P) * (W // P)
patch_dim = P * P * C

print(f"Image size: {H}x{W}x{C}")
print(f"Patch size: {P}x{P}")
print(f"Number of patches: N = ({H}/{P})^2 = {N}")
print(f"Flattened patch dimension: {P}x{P}x{C} = {patch_dim}")
print(f"Embedding dimension: D = {D}")
print(f"Projection matrix E shape: ({patch_dim}, {D})")
print(f"Position embeddings shape: ({N}+1, {D}) = ({N+1}, {D})")
print(f"\nFinal output shape: (batch, {N+1}, {D})")

Let us create a tiny image with recognizable values so we can trace each step by hand.

In [ ]:
# Create a tiny 8x8 grayscale "image" with recognizable values
toy_image = torch.arange(64, dtype=torch.float32).reshape(1, 1, 8, 8)
print("Our 8x8 'image' (values 0-63):")
print(toy_image.squeeze().numpy().astype(int))

In [ ]:
# Step 1: Extract 4 patches of size 4x4
patches = toy_image.unfold(2, P, P).unfold(3, P, P)  # (1, 1, 2, 2, 4, 4)
patches = patches.contiguous().reshape(1, N, patch_dim)  # (1, 4, 16)

print(f"Patches shape: {patches.shape}  ->  (batch=1, N={N}, patch_dim={patch_dim})")
print()
for i in range(N):
    row, col = i // (W // P), i % (W // P)
    print(f"Patch {i} (row={row}, col={col}): {patches[0, i].numpy().astype(int)}")

Notice how Patch 0 contains values 0-7 and 8-15 (the top-left 4x4 block), while Patch 3 contains values 36-39, 44-47, 52-55, 60-63 (the bottom-right block). The patches tile the image perfectly.

Now let us project, prepend [CLS], and add position embeddings.

In [ ]:
# Step 2: Linear projection from patch_dim=16 to D=6
projection = nn.Linear(patch_dim, D, bias=False)
projected = projection(patches)  # (1, 4, 6)
print(f"After projection: {projected.shape}  ->  (batch=1, N={N}, D={D})")
print(f"Weight matrix E shape: {projection.weight.shape}")

In [ ]:
# Step 3: Prepend [CLS] token
cls_token = nn.Parameter(torch.randn(1, 1, D))
tokens = torch.cat([cls_token.expand(1, -1, -1), projected], dim=1)
print(f"After prepending [CLS]: {tokens.shape}  ->  (batch=1, N+1={N+1}, D={D})")

# Step 4: Add position embeddings
pos_embed = nn.Parameter(torch.randn(1, N + 1, D))
output = tokens + pos_embed
print(f"After adding position embeddings: {output.shape}")
print(f"\nFinal output: {output.shape} — ready for the Transformer encoder!")

In [ ]:
#@title 🎧 Listen: Cifar Patches
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_cifar_patches.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 4: Let Us Build It — Component by Component

Now we move from toy examples to real images. We will use CIFAR-10 (32x32 RGB images) and build each component one at a time, with visualizations after each step.

In [ ]:
# Load CIFAR-10
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts to [0, 1] range
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform
)

# Grab a batch of images
dataloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=False)
images, labels = next(iter(dataloader))

# Class names for display
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

print(f"Batch shape: {images.shape}")  # (16, 3, 32, 32)
print(f"First few labels: {[class_names[l] for l in labels[:5].tolist()]}")

### 4.1 Extracting Patches

Our first task: take an image of shape $(C, H, W)$ and chop it into a grid of non-overlapping patches. Each patch is a small $P \times P \times C$ sub-image.

In [ ]:
def extract_patches(images, patch_size):
    """
    Extract non-overlapping patches from a batch of images.

    Args:
        images: Tensor of shape (B, C, H, W)
        patch_size: Side length of each square patch

    Returns:
        patches: Tensor of shape (B, N, patch_dim)
            where N = (H/P)*(W/P) and patch_dim = P*P*C
    """
    B, C, H, W = images.shape
    P = patch_size
    assert H % P == 0 and W % P == 0, \
        f"Image size ({H}x{W}) must be divisible by patch size ({P})"

    n_h, n_w = H // P, W // P
    N = n_h * n_w

    # (B, C, H, W) -> (B, C, n_h, P, n_w, P) -> (B, n_h, n_w, C, P, P) -> (B, N, P*P*C)
    patches = images.reshape(B, C, n_h, P, n_w, P)
    patches = patches.permute(0, 2, 4, 1, 3, 5)
    patches = patches.reshape(B, N, -1)
    return patches

In [ ]:
# Extract patches from our CIFAR-10 images
PATCH_SIZE = 4
patches = extract_patches(images, PATCH_SIZE)

B, N, patch_dim = patches.shape
n_h = n_w = int(N ** 0.5)

print(f"Input images:  {images.shape}")         # (16, 3, 32, 32)
print(f"Patches:       {patches.shape}")         # (16, 64, 48)
print(f"Grid:          {n_h} x {n_w} = {N} patches")
print(f"Patch dim:     {PATCH_SIZE}x{PATCH_SIZE}x3 = {patch_dim}")

In [ ]:
#@title 🎧 Listen: Patch Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_patch_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Look at that! Our 32x32 image is now a sequence of 64 tokens, each with 48 dimensions. Let us visualize what these patches look like.

In [ ]:
# Visualize: original image with its patch grid overlay
sample_idx = 0
sample_image = images[sample_idx]  # (3, 32, 32)
sample_patches = patches[sample_idx]  # (64, 48)

fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(sample_image.permute(1, 2, 0).numpy())
ax.set_title(f"Original: {class_names[labels[sample_idx]]}", fontsize=14)

# Draw grid lines showing patch boundaries
for i in range(1, n_h):
    ax.axhline(y=i * PATCH_SIZE - 0.5, color='white', linewidth=1, alpha=0.8)
    ax.axvline(x=i * PATCH_SIZE - 0.5, color='white', linewidth=1, alpha=0.8)
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
# Display each patch individually in a grid — each is one "token"
fig, axes = plt.subplots(n_h, n_w, figsize=(8, 8))
fig.suptitle("Individual Patches (each is one 'token')", fontsize=14, y=1.02)

for i in range(n_h):
    for j in range(n_w):
        patch_idx = i * n_w + j
        # Reshape flat patch back to image: (48,) -> (3,4,4) -> (4,4,3)
        patch_img = sample_patches[patch_idx].reshape(3, PATCH_SIZE, PATCH_SIZE)
        patch_img = patch_img.permute(1, 2, 0).numpy()
        patch_img = np.clip(patch_img, 0, 1)

        axes[i, j].imshow(patch_img)
        axes[i, j].set_xticks([])
        axes[i, j].set_yticks([])
        axes[i, j].set_aspect('equal')

plt.tight_layout()
plt.show()
print(f"Each patch is a {PATCH_SIZE}x{PATCH_SIZE}x3 = {patch_dim}-dimensional vector.")
print("These are our 'words' — the input tokens to the Transformer.")

In [ ]:
#@title 🎧 Listen: Linear Vs Conv
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_linear_vs_conv.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.2 Flattening and Linear Projection

The patches are already flattened (that is what `extract_patches` returns). Now we need to project them from the raw pixel dimension ($P^2 \cdot C = 48$) into the model's hidden dimension $D$.

There are two equivalent ways to do this:
1. **Manual reshape + `nn.Linear`** — conceptually clear
2. **`nn.Conv2d` with kernel_size=P, stride=P** — computationally efficient

Let us implement both and verify they are equivalent.

In [ ]:
# Configuration
EMBED_DIM = 128  # Model hidden dimension D

# Approach 1: nn.Linear on flattened patches
linear_proj = nn.Linear(patch_dim, EMBED_DIM)

# Project the patches
patch_embeddings_linear = linear_proj(patches)  # (B, N, D)
print(f"Linear projection output: {patch_embeddings_linear.shape}")

In [ ]:
# Approach 2: nn.Conv2d with kernel_size=patch_size, stride=patch_size
# This extracts patches AND projects them in one operation!
conv_proj = nn.Conv2d(
    in_channels=3,
    out_channels=EMBED_DIM,
    kernel_size=PATCH_SIZE,
    stride=PATCH_SIZE,
    bias=True
)

# Apply to original images (not flattened patches)
conv_output = conv_proj(images)  # (B, D, n_h, n_w)
# Reshape to sequence: (B, D, n_h, n_w) -> (B, N, D)
patch_embeddings_conv = conv_output.flatten(2).transpose(1, 2)
print(f"Conv2d projection output: {patch_embeddings_conv.shape}")

Now let us prove these two approaches are mathematically equivalent by copying the weights from one to the other and comparing outputs.

In [ ]:
# Verify equivalence: copy Linear weights into Conv2d
with torch.no_grad():
    # Conv2d weight: (D, C, P, P) = (128, 3, 4, 4)
    # Linear weight: (D, P*P*C)  = (128, 48)
    # Same parameters, just reshaped
    conv_proj.weight.copy_(
        linear_proj.weight.reshape(EMBED_DIM, 3, PATCH_SIZE, PATCH_SIZE)
    )
    conv_proj.bias.copy_(linear_proj.bias)

conv_out = conv_proj(images).flatten(2).transpose(1, 2)
linear_out = linear_proj(patches)

diff = (conv_out - linear_out).abs().max().item()
print(f"Max difference: {diff:.2e}")
print("They are equivalent!" if diff < 1e-5 else "Something went wrong.")

The Conv2d approach is preferred in practice because it is slightly more memory-efficient — it avoids creating an intermediate tensor of flattened patches. But conceptually, it is doing exactly the same thing: extracting non-overlapping patches and multiplying each by a weight matrix.

Now let us visualize how the linear projection reshapes the distribution of patch vectors.

In [ ]:
#@title 🎧 Listen: Pca Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/12_pca_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# PCA to visualize high-dimensional vectors in 2D
from sklearn.decomposition import PCA

raw_vecs = patches[0].detach().numpy()        # (64, 48)
proj_vecs = patch_embeddings_linear[0].detach().numpy()  # (64, 128)

pca_raw = PCA(n_components=2).fit_transform(raw_vecs)
pca_proj = PCA(n_components=2).fit_transform(proj_vecs)

# Color by grid position (top-left=dark, bottom-right=light)
colors = np.arange(N)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(pca_raw[:, 0], pca_raw[:, 1],
                c=colors, cmap='viridis', s=60, edgecolors='k', linewidth=0.5)
axes[0].set_title("Raw Patch Vectors (PCA to 2D)", fontsize=13)
axes[0].set_xlabel("PC 1")
axes[0].set_ylabel("PC 2")

sc = axes[1].scatter(pca_proj[:, 0], pca_proj[:, 1],
                     c=colors, cmap='viridis', s=60, edgecolors='k', linewidth=0.5)
axes[1].set_title("Projected Patch Vectors (PCA to 2D)", fontsize=13)
axes[1].set_xlabel("PC 1")
axes[1].set_ylabel("PC 2")

plt.colorbar(sc, ax=axes[1], label="Patch index (top-left to bottom-right)")
plt.tight_layout()
plt.show()
print("Notice how the linear projection reshapes the distribution —")
print("it learns to spread patches in ways useful for attention.")

In [ ]:
#@title 🎧 Listen: Cls And Position
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/13_cls_and_position.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.3 The [CLS] Token

In the original ViT paper, a special learnable **[CLS] token** is prepended to the patch sequence. This token does not correspond to any image patch. Instead, it serves as a **global summary** of the entire image.

Why? At the end of the Transformer encoder, the representation at the [CLS] position is used for classification. Through self-attention, this token can attend to all patches and aggregate their information.

In [ ]:
# The [CLS] token is a learnable parameter vector of dimension D
cls_token = nn.Parameter(torch.randn(1, 1, EMBED_DIM))

print(f"[CLS] token shape: {cls_token.shape}")
print(f"Initialized to random values (will be learned during training)")
print(f"First few values: {cls_token.data[0, 0, :8].numpy().round(3)}")

In [ ]:
# Prepend [CLS] to the sequence of patch embeddings
batch_size = patch_embeddings_linear.shape[0]
cls_tokens = cls_token.expand(batch_size, -1, -1)  # (B, 1, D)

# Concatenate: [CLS] + patch embeddings
tokens = torch.cat([cls_tokens, patch_embeddings_linear], dim=1)

print(f"Patch embeddings shape: {patch_embeddings_linear.shape}")  # (B, 64, 128)
print(f"After prepending [CLS]:  {tokens.shape}")  # (B, 65, 128)
print(f"\nSequence length went from {N} to {N+1}")
print(f"Token 0 = [CLS] (global summary)")
print(f"Tokens 1-{N} = patch embeddings")

The [CLS] token starts as random noise, but during training it learns to become a powerful aggregator. Through self-attention, it can "query" every patch in the image and compose a holistic representation. This is a direct analog of BERT's [CLS] token in NLP.

### 4.4 Position Embeddings

Without position information, the Transformer would treat our patch sequence as a **bag of patches** — it would have no idea that Patch 0 is in the top-left and Patch 63 is in the bottom-right. Self-attention is permutation-equivariant by design.

Position embeddings fix this. We add a learnable vector to each token that encodes its position in the sequence.

In [ ]:
# Learnable position embeddings: one per token (including [CLS])
num_tokens = N + 1  # 64 patches + 1 [CLS] = 65
pos_embedding = nn.Parameter(torch.randn(1, num_tokens, EMBED_DIM))

print(f"Position embeddings shape: {pos_embedding.shape}")  # (1, 65, 128)
print(f"One embedding for [CLS] + one for each of the {N} patches")

In [ ]:
# Add position embeddings to the token sequence (element-wise)
embedded_tokens = tokens + pos_embedding

print(f"Before position embeddings: {tokens.shape}")
print(f"After position embeddings:  {embedded_tokens.shape}")
print(f"\nShape unchanged — position info is encoded by shifting")
print(f"each token's vector in embedding space.")

In [ ]:
#@title 🎧 Listen: Pos Embed Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/14_pos_embed_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Now let us visualize the position embeddings. Even though they are randomly initialized (untrained), the cosine similarity matrix will show us what structure we expect to emerge after training.

In [ ]:
# Cosine similarity between all pairs of position embeddings
pos_emb = pos_embedding.squeeze(0).detach()  # (65, 128)
pos_norm = pos_emb / pos_emb.norm(dim=1, keepdim=True)
cos_sim = pos_norm @ pos_norm.T  # (65, 65)

fig, ax = plt.subplots(figsize=(7, 6))
im = ax.imshow(cos_sim.numpy(), cmap='coolwarm', vmin=-1, vmax=1)
ax.set_title("Position Embedding Cosine Similarity\n(including [CLS] at index 0)", fontsize=12)
ax.set_xlabel("Token position")
ax.set_ylabel("Token position")
plt.colorbar(im, ax=ax, shrink=0.8)
plt.tight_layout()
plt.show()

Let us look at something more interesting: pick a reference patch and see how similar it is to all other patches in the 2D grid.

In [ ]:
# Similarity from one reference patch to all others, reshaped to 2D grid
patch_pos = pos_emb[1:]  # Remove [CLS], shape: (64, 128)
patch_norm = patch_pos / patch_pos.norm(dim=1, keepdim=True)
patch_cos_sim = patch_norm @ patch_norm.T  # (64, 64)

ref_idx = N // 2 + n_w // 2  # roughly center patch
ref_sim = patch_cos_sim[ref_idx].reshape(n_h, n_w).numpy()

fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(ref_sim, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_title(f"Similarity of center patch (idx={ref_idx}) to all patches\n"
             f"(Random init — no spatial structure yet)", fontsize=12)
ax.set_xlabel("Patch column")
ax.set_ylabel("Patch row")
plt.colorbar(im, ax=ax, shrink=0.8)
plt.tight_layout()
plt.show()

print("Right now this looks like noise — the position embeddings are random.")
print("After training, nearby patches will have HIGH similarity,")
print("revealing learned 2D structure!")

In [ ]:
#@title 🎧 Listen: Todo1
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/15_todo1.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 5: Your Turn

Time to test your understanding! Fill in the missing code to build the complete `PatchEmbedding` module.

### TODO 1: Complete the PatchEmbedding Forward Pass

In [ ]:
class PatchEmbedding(nn.Module):
    """
    Converts an image into a sequence of patch embeddings
    with [CLS] token and position embeddings.
    """
    def __init__(self, img_size=32, patch_size=4, in_channels=3, embed_dim=128):
        super().__init__()
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_dim = patch_size * patch_size * in_channels

        # Linear projection from flattened patch to embedding
        self.projection = nn.Linear(self.patch_dim, embed_dim)

        # Learnable [CLS] token
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))

        # Learnable position embeddings
        self.pos_embedding = nn.Parameter(
            torch.randn(1, self.num_patches + 1, embed_dim)
        )

In [ ]:
    # (continued from PatchEmbedding class above)
    def forward(self, x):
        """Forward pass: (B, C, H, W) -> (B, num_patches+1, embed_dim)"""
        B, C, H, W = x.shape
        P = self.patch_size

        # TODO: Fill in the forward pass
        # Hint 1: x.reshape(B, C, H//P, P, W//P, P)
        #   -> permute to (B, H//P, W//P, C, P, P)
        #   -> reshape to (B, N, patch_dim)
        # Hint 2: Project with self.projection
        # Hint 3: self.cls_token.expand(B, -1, -1)
        #   -> torch.cat([cls_tokens, ...], dim=1)
        # Hint 4: Add self.pos_embedding

        output = ???  # Your implementation here

        return output

#### Verification Cell for TODO 1

Run this cell to check your implementation. Do not modify it.

In [ ]:
# Verification for TODO 1
try:
    model_test = PatchEmbedding(
        img_size=32, patch_size=4, in_channels=3, embed_dim=128
    )
    test_input = torch.randn(2, 3, 32, 32)
    test_output = model_test(test_input)

    expected_shape = (2, 65, 128)  # batch=2, 64+1 tokens, D=128
    assert test_output.shape == expected_shape, \
        f"Shape mismatch: got {test_output.shape}, expected {expected_shape}"

    # Check gradients flow
    loss = test_output.sum()
    loss.backward()
    assert model_test.projection.weight.grad is not None
    assert model_test.cls_token.grad is not None
    assert model_test.pos_embedding.grad is not None

    print("TODO 1 PASSED!")
    print(f"   Output shape: {test_output.shape}")
    print(f"   Gradients flow to all learnable parameters.")
except Exception as e:
    print(f"TODO 1 FAILED: {e}")
    print("   Review your implementation and try again.")

In [ ]:
#@title 🎧 Listen: Todo2
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/16_todo2.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### TODO 2: Patch Extraction with Conv2d

Implement the patch extraction using `nn.Conv2d` instead of manual reshaping. The key insight: a Conv2d with `kernel_size=P` and `stride=P` extracts non-overlapping patches and projects them simultaneously.

In [ ]:
class PatchEmbeddingConv(nn.Module):
    """
    Same as PatchEmbedding, but uses Conv2d for extraction + projection.
    """
    def __init__(self, img_size=32, patch_size=4, in_channels=3, embed_dim=128):
        super().__init__()
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2

        # ============================================
        # TODO: Create a Conv2d that extracts patches and projects
        # them in one operation.
        # Hint: kernel_size and stride should both be patch_size.
        #       out_channels should be embed_dim.
        # ============================================
        self.projection = ???  # Your Conv2d layer here

        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.pos_embedding = nn.Parameter(
            torch.randn(1, self.num_patches + 1, embed_dim)
        )

In [ ]:
    # (continued from PatchEmbeddingConv class above)
    def forward(self, x):
        B = x.shape[0]

        # ============================================
        # TODO: Apply Conv2d, reshape to (B, N, D),
        # prepend [CLS], add position embeddings.
        #
        # Hint: Conv2d output: (B, embed_dim, n_h, n_w)
        #   -> .flatten(2) -> (B, embed_dim, N)
        #   -> .transpose(1, 2) -> (B, N, embed_dim)
        # ============================================
        output = ???  # Your implementation here

        return output

#### Verification Cell for TODO 2

In [ ]:
# Verification for TODO 2
try:
    model_conv = PatchEmbeddingConv(
        img_size=32, patch_size=4, in_channels=3, embed_dim=128
    )
    test_input = torch.randn(2, 3, 32, 32)
    test_output = model_conv(test_input)

    expected_shape = (2, 65, 128)
    assert test_output.shape == expected_shape, \
        f"Shape mismatch: got {test_output.shape}, expected {expected_shape}"

    conv_layer = model_conv.projection
    assert isinstance(conv_layer, nn.Conv2d), "projection should be nn.Conv2d"
    assert conv_layer.kernel_size == (4, 4), \
        f"kernel_size should be (4,4), got {conv_layer.kernel_size}"
    assert conv_layer.stride == (4, 4), \
        f"stride should be (4,4), got {conv_layer.stride}"

    print("TODO 2 PASSED!")
    print(f"   Output shape: {test_output.shape}")
    print(f"   Conv2d kernel: {conv_layer.kernel_size}, stride: {conv_layer.stride}")
except Exception as e:
    print(f"TODO 2 FAILED: {e}")
    print("   Review your Conv2d parameters and try again.")

In [ ]:
#@title 🎧 Listen: Complete Module
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/17_complete_module.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 6: Putting It All Together

Here is the complete, clean `PatchEmbedding` class — our reference implementation that combines everything we have built.

In [ ]:
class PatchEmbeddingComplete(nn.Module):
    """
    Complete patch embedding module for Vision Transformers.

    Takes an image tensor and produces a sequence of patch embeddings
    with a prepended [CLS] token and learned position embeddings.
    """
    def __init__(self, img_size=32, patch_size=4, in_channels=3, embed_dim=128):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2

        # Conv2d-based patch extraction + projection
        self.projection = nn.Conv2d(
            in_channels, embed_dim,
            kernel_size=patch_size, stride=patch_size
        )
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.pos_embedding = nn.Parameter(
            torch.randn(1, self.num_patches + 1, embed_dim)
        )
        self._init_weights()

In [ ]:
    # (continued from PatchEmbeddingComplete)
    def _init_weights(self):
        """Xavier uniform initialization for stable training."""
        nn.init.xavier_uniform_(self.projection.weight.view(
            self.projection.weight.size(0), -1
        ))
        nn.init.zeros_(self.projection.bias)
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        nn.init.trunc_normal_(self.pos_embedding, std=0.02)

    def forward(self, x):
        B = x.shape[0]

        # Extract patches and project via Conv2d: (B,C,H,W) -> (B,D,n_h,n_w)
        patch_embeddings = self.projection(x)

        # Reshape to sequence: (B,D,n_h,n_w) -> (B,N,D)
        patch_embeddings = patch_embeddings.flatten(2).transpose(1, 2)

        # Prepend [CLS] token
        cls_tokens = self.cls_token.expand(B, -1, -1)
        tokens = torch.cat([cls_tokens, patch_embeddings], dim=1)

        # Add position embeddings
        return tokens + self.pos_embedding

Now let us feed a batch of real CIFAR-10 images through and trace the shape at every stage.

In [ ]:
# Instantiate and run
model = PatchEmbeddingComplete(
    img_size=32, patch_size=4, in_channels=3, embed_dim=128
)
output = model(images)

# Shape trace
print("=" * 60)
print("Shape Trace Through PatchEmbeddingComplete")
print("=" * 60)
with torch.no_grad():
    x = images
    print(f"Input images:            {x.shape}")
    proj = model.projection(x)
    print(f"After Conv2d:            {proj.shape}")
    flat = proj.flatten(2).transpose(1, 2)
    print(f"After flatten+transpose: {flat.shape}")
    cls = model.cls_token.expand(x.shape[0], -1, -1)
    cat = torch.cat([cls, flat], dim=1)
    print(f"After [CLS] prepend:     {cat.shape}")
    final = cat + model.pos_embedding
    print(f"After pos embedding:     {final.shape}")
print("=" * 60)

In [ ]:
# Parameter count
total_params = sum(p.numel() for p in model.parameters())

print(f"\nFinal output: {output.shape}")
print(f"  {output.shape[0]} images, each as {output.shape[1]} tokens "
      f"(1 [CLS] + {model.num_patches} patches), dim {output.shape[2]}")
print(f"\nTotal parameters: {total_params:,}")
print(f"  Conv2d projection: "
      f"{sum(p.numel() for p in model.projection.parameters()):,}")
print(f"  [CLS] token:       {model.cls_token.numel():,}")
print(f"  Position embed:    {model.pos_embedding.numel():,}")
print(f"\nThis is ready to feed into a Transformer encoder!")

In [ ]:
#@title 🎧 Listen: Final Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/18_final_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 7: Final Output

Let us create a comprehensive visualization of our complete patch embedding pipeline.

In [ ]:
# Select a visually interesting image
sample_idx = 7  # Try different indices!
sample_image = images[sample_idx]
sample_label = class_names[labels[sample_idx]]

model.eval()
with torch.no_grad():
    sample_output = model(sample_image.unsqueeze(0))

print(f"Selected image: {sample_label}")
print(f"Output shape: {sample_output.shape}")

**Panel 1:** Original image with patch grid overlay.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
img_np = np.clip(sample_image.permute(1, 2, 0).numpy(), 0, 1)
ax.imshow(img_np)

# Draw patch grid
for i in range(1, n_h):
    ax.axhline(y=i * PATCH_SIZE - 0.5, color='yellow', linewidth=1.5, alpha=0.9)
    ax.axvline(x=i * PATCH_SIZE - 0.5, color='yellow', linewidth=1.5, alpha=0.9)

# Label select patches
for i in range(n_h):
    for j in range(n_w):
        if (i + j) % 3 == 0:
            num = i * n_w + j
            ax.text(j * PATCH_SIZE + PATCH_SIZE/2,
                    i * PATCH_SIZE + PATCH_SIZE/2,
                    str(num), ha='center', va='center',
                    fontsize=6, color='yellow', fontweight='bold')

ax.set_title(f"Original Image ({n_h}x{n_w} = {N} patches)", fontsize=13)
ax.set_xticks([]); ax.set_yticks([])
plt.show()

**Panel 2:** Individual patches displayed in a grid.

In [ ]:
fig, axes = plt.subplots(n_h, n_w, figsize=(8, 8))
fig.suptitle("64 Patches — Each Becomes One Token", fontsize=14, y=1.02)

sp = extract_patches(sample_image.unsqueeze(0), PATCH_SIZE)[0]

for i in range(n_h):
    for j in range(n_w):
        idx = i * n_w + j
        p_img = sp[idx].reshape(3, PATCH_SIZE, PATCH_SIZE)
        p_img = np.clip(p_img.permute(1, 2, 0).numpy(), 0, 1)
        axes[i, j].imshow(p_img)
        axes[i, j].set_xticks([])
        axes[i, j].set_yticks([])

plt.tight_layout()
plt.show()

**Panel 3:** Position embedding cosine similarity heatmaps. We show how each patch's position embedding relates to all other patches.

In [ ]:
# Compute patch-only position embedding similarities
pos_emb_final = model.pos_embedding.squeeze(0).detach()
patch_pos_emb = pos_emb_final[1:]  # (64, 128) — exclude [CLS]
patch_pos_norm = patch_pos_emb / patch_pos_emb.norm(dim=1, keepdim=True)
patch_cos_sim = patch_pos_norm @ patch_pos_norm.T

# 8 reference patches at diverse grid positions
ref_positions = [0, 3, 7, 24, 31, 39, 56, 63]
ref_labels = ["Top-left", "Top-mid", "Top-right", "Mid-left",
              "Center", "Mid-right", "Bot-left", "Bot-right"]

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(16, 7))
fig.suptitle(
    "Position Embedding Similarities\n"
    "(Random init — 2D structure emerges after training)",
    fontsize=14, y=1.05
)

for ax_idx, (ref_idx, label) in enumerate(zip(ref_positions, ref_labels)):
    row, col = ax_idx // 4, ax_idx % 4
    sim_map = patch_cos_sim[ref_idx].reshape(n_h, n_w).numpy()

    im = axes[row, col].imshow(sim_map, cmap='coolwarm', vmin=-1, vmax=1)
    axes[row, col].set_title(f"{label}\n(patch {ref_idx})", fontsize=10)
    axes[row, col].set_xticks([])
    axes[row, col].set_yticks([])

    # Mark the reference patch with a star
    r, c = ref_idx // n_w, ref_idx % n_w
    axes[row, col].plot(c, r, 'k*', markersize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Final summary
print("=" * 60)
print("  PIPELINE SUMMARY")
print("=" * 60)
print(f"  Input:   {sample_image.shape} image ({sample_label})")
print(f"  Patches: {N} patches of size {PATCH_SIZE}x{PATCH_SIZE}x3")
print(f"  Patch dim: {PATCH_SIZE * PATCH_SIZE * 3}")
print(f"  Embed dim: {EMBED_DIM}")
print(f"  Output:  {sample_output.shape}")
print(f"    Token 0:     [CLS] (global summary)")
print(f"    Tokens 1-{N}: patch embeddings with position info")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")
print("=" * 60)
print()
print("Congratulations! You have built the input pipeline")
print("for a Vision Transformer!")
print()
print("This sequence of 65 token embeddings is exactly what gets")
print("fed into the Transformer encoder. In Notebook 2, we will")
print("build multi-head self-attention to let these tokens")
print("communicate with each other.")

In [ ]:
#@title 🎧 Listen: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/19_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 8: Reflection and Next Steps

### Reflection Questions

Take a moment to think about these before moving on:

1. **Patch size trade-off:** What happens to the number of patches if we use 8x8 patches instead of 4x4 on our 32x32 images? We would get $(32/8)^2 = 16$ patches instead of 64. Fewer patches means a shorter sequence, which means **quadratically less computation** in self-attention (attention is $O(N^2)$). But each patch covers more of the image, so we lose fine-grained spatial detail. This is a fundamental resolution-vs-computation trade-off.

2. **Why not raw pixels?** Why do we use a learnable linear projection instead of feeding raw pixel vectors directly to the Transformer? Raw pixel values live in a space that is not aligned with the features the Transformer needs. The linear projection learns to extract useful initial features — edges, color gradients, textures — that give the Transformer a better starting representation. It is also a dimensionality change: we can project 48-dimensional raw patches into whatever dimension $D$ we want.

3. **1D vs 2D position embeddings:** Our position embeddings are 1D — one vector per position in the flattened sequence. The model has to **learn** that position 0 and position 1 are horizontally adjacent, while position 0 and position 8 are vertically adjacent. Remarkably, it does learn this! An alternative is 2D position embeddings that explicitly encode (row, column) — the original ViT paper tried this and found it performs similarly, suggesting that 1D learnable embeddings are sufficient.

### Optional Challenges

If you want to go deeper, try these.

In [ ]:
# Challenge 1: Try different patch sizes
# Uncomment and run to see how patch size affects the grid

# for ps in [2, 4, 8, 16]:
#     if 32 % ps == 0:
#         n = (32 // ps) ** 2
#         dim = ps * ps * 3
#         m = PatchEmbeddingComplete(img_size=32, patch_size=ps, embed_dim=128)
#         out = m(images[:1])
#         print(f"Patch size {ps:2d}: {n:4d} patches, "
#               f"patch_dim={dim:4d}, output={out.shape}")

In [ ]:
# Challenge 2: Implement 2D sinusoidal position embeddings
# In the original Transformer (Vaswani et al.), position embeddings used
# sine and cosine functions. Can you adapt that to 2D?
#
# Hint: Create separate sin/cos embeddings for row and column,
# then concatenate. Each position (r, c) gets:
#   pos_embed(r,c) = [sin_embed(r); cos_embed(r);
#                     sin_embed(c); cos_embed(c)]
#
# def get_2d_sinusoidal_pos_embed(embed_dim, grid_size):
#     """
#     Args:
#         embed_dim: Total dimension (must be divisible by 4)
#         grid_size: Patches along each axis
#     Returns:
#         pos_embed: shape (grid_size*grid_size, embed_dim)
#     """
#     pass  # Your implementation here!

### What is Next?

In **Notebook 2: Multi-Head Self-Attention for Images**, we will build the core mechanism that makes Transformers so powerful. Our 65-token sequence from this notebook will become the input, and attention will let each token "look at" every other token to build richer representations.

The key question we will answer: **How does a patch in the bottom-right corner of an image learn that it is related to a patch in the top-left corner?** The answer is self-attention — and we will build it from scratch.

See you there!